<a href="https://colab.research.google.com/github/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch/InferenceOptimizationAndTTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference Optimization and Test Time Augmentation (TTA)

This is the inference pipeline for the model trained in  [A complex yet simple efficient training pipeline for CIFAR-10](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch/ComplexYetSimpleTrainingPipeline.ipynb). 

In this Notebook you will learn about simple inference optimization you can do while serving pytorch models.

For more advanced information, please check [TorchScript](https://docs.pytorch.org/docs/stable/jit.html) and [onnx](https://onnxruntime.ai/).

The full inference script is available in [inference_optimization_and_tta.py](./inference_optimization_and_tta.py).

---

| All     | [advanced_pytorch/](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch) |
|---------|-- |
| Current | [Inference Optimization And TTA](https://github.com/Tensor-Reloaded/AI-Learning-Hub/blob/main/resources/advanced_pytorch/InferenceOptimizationAndTTA.ipynb) |